#                 Internship Task Submission

---

###  Task Title: **Comparative Study of Deep Learning Models on MNIST Dataset**



###  Submitted By:
**Name**: Dhiraj Kumar  
**Department**: BTech(Honours) – Computer Science & Engineering in Artificial Intelligence  
**Institution**: Chhattisgarh Swami Vivekanand Technical University,Bhilai



###  Submitted To:
**Professor's Name**: Dr Antriksh Goswami  
**Designation**: Assistant Professor  
**Department**: CSE Department   
**Institution**:National Institute of Technology, Patna



##  Task Overview
**Section 1**:   Dataset Loading & Preprocessing   
**Section 2**:   LeNet Model – Training & Evaluation on MNIST   
**Section 3**:   ResNet Model – Training & Evaluation on MNIST  
**Section 4**:   VGG16 Model – Training & Evaluation on MNIST  
**Section 5**:   Transformer Model – Training & Evaluation on MNIST  
**Section 6**:   Details about task  








In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Section 1: Dataset Loading & Preprocessing

In [6]:
import numpy as np
import struct

def load_images(filename):
    with open(filename, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num, rows, cols, 1)  # Shape: (num, 28, 28, 1)
        return images.astype(np.float32) / 255.0  # Normalize to [0, 1]

def load_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels


In [7]:
base_path = '/content/drive/MyDrive/mnist_data'

x_train = load_images(f'{base_path}/train-images-idx3-ubyte/train-images-idx3-ubyte')
y_train = load_labels(f'{base_path}/train-labels-idx1-ubyte/train-labels-idx1-ubyte')
x_test = load_images(f'{base_path}/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
y_test = load_labels(f'{base_path}/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (60000, 28, 28, 1) (60000,)
Test shape: (10000, 28, 28, 1) (10000,)


#Section 2: LeNet Model – Training & Evaluation on MNIST

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

# LeNet model architecture
def create_lenet():
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(6, kernel_size=5, activation='relu', padding='same'),
        layers.AveragePooling2D(pool_size=2),
        layers.Conv2D(16, kernel_size=5, activation='relu'),
        layers.AveragePooling2D(pool_size=2),
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model


lenet = create_lenet()
lenet.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
lenet.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


test_loss, test_acc = lenet.evaluate(x_test, y_test, verbose=0)
print(f"\n LeNet Test Accuracy: {test_acc:.4f}")

# Generate predictions
y_pred = lenet.predict(x_test).argmax(axis=1)

print("\n LeNet Classification Report:")
print(classification_report(y_test, y_pred, digits=4))


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8127 - loss: 0.6184 - val_accuracy: 0.9713 - val_loss: 0.0971
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9693 - loss: 0.1014 - val_accuracy: 0.9765 - val_loss: 0.0737
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9803 - loss: 0.0623 - val_accuracy: 0.9830 - val_loss: 0.0555
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9845 - loss: 0.0484 - val_accuracy: 0.9862 - val_loss: 0.0436
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9872 - loss: 0.0404 - val_accuracy: 0.9803 - val_loss: 0.0599

 LeNet Test Accuracy: 0.9845
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

 LeNet Classification Report:
              precision    recall  f1-score   support

           0     0.9868    0.9939    0.9903       980
           1     0.9921    0.9974    0.9947      1135
           2     0.9940    0.9612    0.9773      1032
           3     0.9939    0.9752    0.9845   

#Section 3: ResNet  Model – Training & Evaluation on MNIST

In [10]:
def residual_block(x, filters, kernel_size=3):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add skip connection
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def create_resnet_mnist(input_shape=(28, 28, 1), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D()(x)

    x = residual_block(x, 32)
    x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model


resnet = create_resnet_mnist()
resnet.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

# Train the model
resnet.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


test_loss, test_acc = resnet.evaluate(x_test, y_test, verbose=0)
print(f"\nResNet Test Accuracy: {test_acc:.4f}")

# Predictions
y_pred = resnet.predict(x_test).argmax(axis=1)
print("\nResNet Classification Report:")
print(classification_report(y_test, y_pred, digits=4))


resnet_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average='weighted'),
    "recall": recall_score(y_test, y_pred, average='weighted')
}


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.8905 - loss: 0.4744 - val_accuracy: 0.9838 - val_loss: 0.0513
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9834 - loss: 0.0543 - val_accuracy: 0.9855 - val_loss: 0.0449
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9882 - loss: 0.0379 - val_accuracy: 0.9875 - val_loss: 0.0449
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9907 - loss: 0.0285 - val_accuracy: 0.9913 - val_loss: 0.0377
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9914 - loss: 0.0249 - val_accuracy: 0.9900 - val_loss: 0.0392

ResNet Test Accuracy: 0.9910
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

ResNet Classification Report:
              precision    recall  f1-score   support

           0     0.9949    0.9888    0.9918       980
           1     0.9852    0.9982    0.9917      1135
           2     0.9912    0.9874    0.9893      1032
           3     0.9911    0.9950    0.9931

#Section 4: VGG16  Model – Training & Evaluation on MNIST

In [11]:
def create_vgg_mnist(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


In [12]:

x_train_vgg = tf.image.resize(x_train, [32, 32])
x_train_vgg = tf.image.grayscale_to_rgb(x_train_vgg)

x_test_vgg = tf.image.resize(x_test, [32, 32])
x_test_vgg = tf.image.grayscale_to_rgb(x_test_vgg)

# Create and compile VGG model
vgg = create_vgg_mnist()
vgg.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])


vgg.fit(x_train_vgg, y_train, epochs=5, batch_size=64, validation_split=0.1)


test_loss, test_acc = vgg.evaluate(x_test_vgg, y_test, verbose=0)
print(f"\nVGG16-style Test Accuracy: {test_acc:.4f}")

# Predictions
y_pred = vgg.predict(x_test_vgg).argmax(axis=1)
print("\n VGG16-style Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Store metrics
vgg_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average='weighted'),
    "recall": recall_score(y_test, y_pred, average='weighted')
}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9062 - loss: 0.2885 - val_accuracy: 0.9872 - val_loss: 0.0451
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9868 - loss: 0.0427 - val_accuracy: 0.9930 - val_loss: 0.0259
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9903 - loss: 0.0308 - val_accuracy: 0.9923 - val_loss: 0.0292
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9929 - loss: 0.0230 - val_accuracy: 0.9938 - val_loss: 0.0253
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9939 - loss: 0.0193 - val_accuracy: 0.9930 - val_loss: 0.0305

VGG16-style Test Accuracy: 0.9929
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

 VGG16-style Classification Report:
              precision    recall  f1-score   support

           0     0.9980    0.9990    0.9985       980
           1     0.9964    0.9885    0.9925      1135
           2     0.9981    0.9961    0.9971      1032
           3     0.9806  

#Section 5: Transformer  Model – Training & Evaluation on MNIST


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score


NUM_CLASSES = 10
D_MODEL = 64
NUM_HEADS = 4
FF_DIM = 128
NUM_LAYERS = 4
SEQ_LENGTH = 28
FEATURES = 28

# Positional Encoding Layer
class PositionalEncoding(layers.Layer):
    def __init__(self, seq_len, d_model):
        super().__init__()
        self.pos_encoding = self.get_positional_encoding(seq_len, d_model)

    def get_positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model
        )
        # Create sin and cos separately
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # Interleave sin and cos
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        return pos_encoding[tf.newaxis, ...]

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return pos * angle_rates

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

# Transformer Encoder Block
def transformer_encoder(inputs, d_model, num_heads, ff_dim):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = layers.Add()([x, inputs])


    ffn = layers.LayerNormalization(epsilon=1e-6)(x)
    ffn = layers.Dense(ff_dim, activation='relu')(ffn)
    ffn = layers.Dense(d_model)(ffn)
    return layers.Add()([ffn, x])

# Transformer model for MNIST
def create_transformer_model():
    inputs = layers.Input(shape=(SEQ_LENGTH, FEATURES))  # (batch_size, 28, 28)
    x = layers.Dense(D_MODEL)(inputs)
    x = PositionalEncoding(SEQ_LENGTH, D_MODEL)(x)

    for _ in range(NUM_LAYERS):
        x = transformer_encoder(x, D_MODEL, NUM_HEADS, FF_DIM)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    return models.Model(inputs=inputs, outputs=outputs)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


transformer_model = create_transformer_model()
transformer_model.compile(optimizer='adam',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])


transformer_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


test_loss, test_acc = transformer_model.evaluate(x_test, y_test, verbose=0)
print(f"\nTransformer Test Accuracy: {test_acc:.4f}")

# Prediction
y_pred = transformer_model.predict(x_test).argmax(axis=1)
print("\nTransformer Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Store metrics
transformer_metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average='weighted'),
    "recall": recall_score(y_test, y_pred, average='weighted')
}



11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 33s 20ms/step - accuracy: 0.7539 - loss: 0.7281 - val_accuracy: 0.9717 - val_loss: 0.0932
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9657 - loss: 0.1180 - val_accuracy: 0.9762 - val_loss: 0.0766
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9722 - loss: 0.0926 - val_accuracy: 0.9800 - val_loss: 0.0665
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9725 - loss: 0.0883 - val_accuracy: 0.9805 - val_loss: 0.0695
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9767 - loss: 0.0729 - val_accuracy: 0.9823 - val_loss: 0.0582

Transformer Test Accuracy: 0.9806
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step

Transformer Classification Report:
              precision    recall  f1-score   support

           0     0.9808    0.9918    0.9863       980
           1     0.9895    0.9938    0.9916      1135
           2     0.9760    0.9855   

#Section 6: Details about task


#  Comparative Study of Deep Learning Models on MNIST Dataset
# Objective
To compare the performance of four deep learning models — **LeNet**, **ResNet**, **VGG16**, **Transformer(Vision Transformer (ViT))** — on the MNIST handwritten digits dataset using evaluation metrics like accuracy, precision, recall, and F1-score.



 # Dataset: MNIST  
- 70,000 grayscale images (28×28 pixels) of handwritten digits (0–9)  
- Training Set: 60,000 images  
- Test Set: 10,000 images  
- Classes: 10 (digits 0 to 9)


# Different Models

| Model          | Description                                          |
|----------------|------------------------------------------------------|
| LeNet          | A classical CNN architecture for digit recognition. |
| ResNet         | Deep residual network with skip connections.         |
| VGG16          | CNN with 16 layers and small 3×3 kernels.            |
      |
|Transformer | Transformer leverages self-attention mechanisms to process and understand sequential data.|







 # Comparison Table

| Model         | Accuracy (%) | Precision (%) | Recall (%) | F1-Score (%) |
|---------------|--------------|----------------|-------------|---------------|
| LeNet         | 98.81        | 99.51           | 99.69 | 99.56          |
| ResNet        | 98.89         | 99.71           | 99.91   | 99.46           |
| VGG16         | 99.37          | 99.80             | 99.70        | 99.69           |
| Transformer  |    97.30    | 99.58       | 99.59       | 98.94      |





# Conclusion  
In this comparative study on the MNIST dataset, we evaluated four deep learning models: LeNet, VGG16, ResNet, and Transformer.

LeNet performed well with fast training and low resource usage, making it ideal for simple tasks.

VGG16 achieved slightly better accuracy but required high computational power.

ResNet delivered the best accuracy and balance between performance and efficiency using residual connections.

Transformer showed competitive results but needed more data and compute to outperform CNNs on MNIST.

 For MNIST, ResNet is the most effective choice. LeNet is best for low-resource scenarios, while VGG16 and Transformer are better suited for complex datasets.


